<a href="https://colab.research.google.com/github/rodrigoromanguzman/Actividades_Aprendizaje-/blob/main/Redneuronal_Equipo08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 7**
###**Red Neuronal Artificial - Perceptrón Multicapa : Multilayer Perceptrón (MLP)**


**Nombres y matrículas de los integrantes del equipo:**

*   Adrián Alejandro Dávila González - A01039334
*   Juan Antonio Melendres Villa - A00369017
*   Andrea Margarita Osorio González - A01104776
*   Rodrigo Ildefonso Román Guzmán - A01794225
*   Jeanette Ríos Martínez - A01688888

En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [1]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.dummy import DummyRegressor

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, LabelEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor

from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import cross_validate, RepeatedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance

#**Ejercicio-1.** 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
DIR = '/content/drive/MyDrive/Maestría/Segundo Trimestre/Inteligencia Artificial y Aprendizaje Automático/Semana 7'
os.chdir(DIR)

FileNotFoundError: ignored

In [ ]:
#Lectura del archivo .csv para crear el dataframe
Fb_df = pd.read_csv('dataset_Facebook.csv', sep=';', header= 'infer')
Fb_df.head()

In [ ]:
#Ajuste del tipo de dato de la variable 'Type' para cambiar de String a Numérico como el resto del conjunto
#-Importante debido a que es una de las variables de entrada a considerar para el modelo

#Debido al uso del LabelEncoder(), las nuevas etiquetas quedan de la siguiente forma:
# 0 - Link
# 1 - Photo
# 2 - Status
# 3 - Video

lab_enc= LabelEncoder()
Fb_df['Type']= lab_enc.fit_transform(Fb_df['Type'])
Fb_df.head()

In [ ]:
#Definición de los nombres correctos de las columnas
col_names= {'Lifetime People who have liked your Page and engaged with your post' : 'LPE'}

#Renombrado de las columnas a su valor correcto
Fb_df.rename(columns= col_names, inplace=True)
Fb_df.head()

In [ ]:
#Generación de Dataframe mis_datos
mis_datos=Fb_df[['Category', 'Page total likes', 'Type', 'Post Month', 'Post Hour', 'Post Weekday', 'Paid','LPE']]
mis_datos.head()

In [ ]:
#Separación de variables y generación de dataframes de predictores y respuesta
X = mis_datos[['Category', 'Page total likes', 'Type', 'Post Month', 'Post Hour', 'Post Weekday', 'Paid']]
y = mis_datos[['LPE']]

In [ ]:
#Conversión del tipo de dato de la variable de salida para ajustarse a flotante en lugar de entero
y= y.astype(np.float64)
y.info()

#**Ejercicio-2.**

In [ ]:
#Partición de los datos en:
# - Conjunto de entrenamiento en 80% (400 datos)
# - Conjunto de prueba en 20% (100 datos)

#Definición y ejecución de la función 'train_test_split()'
X_train_v, X_test, y_train_v, y_test = train_test_split(X, y, test_size=0.2, shuffle= True)
print('\033[1m' + 'Forma de X_train_v:' + '\033[0m', X_train_v.shape)
print('\033[1m' + 'Forma de y_train_v:' + '\033[0m', y_train_v.shape)
print('\033[1m' + 'Forma de X_test:' + '\033[0m', X_test.shape)
print('\033[1m' + 'Forma de y_test:' + '\033[0m', y_test.shape)

In [ ]:
X_train_v.info()

#**Ejercicio-3.**

In [ ]:
# Definición de función para calcular la raíz cuadrada del error cuadrático medio (RMSE)
def mi_RMSE(y_val,yhatVal):
  rmse = pow(np.mean(np.square(y_val-yhatVal), axis=0),0.5)
  return rmse

# Definición de función para calcular el error absoluto medio (MAE)
def mi_MAE(y_val,yhatVal):
  mae = np.mean(np.absolute(y_val-yhatVal), axis=0)
  return mae

# Definición de función para calcular el error porcentual absoluto medio (MAPE)
def mi_MAPE(y_val,yhatVal):
  epsilon= np.finfo(float).eps
  mape = np.mean(np.absolute((y_val-yhatVal)/(y_val+epsilon)), axis=0)*100
  return mape

#**Ejercicio-4.**

In [ ]:
#Separación de variables en conjuntos de atributos en base a tipo de dato
categorical_attributes = ['Category', 'Type', 'Paid']
numerical_attributes = ['Page total likes', 'Post Month', 'Post Hour', 'Post Weekday']

In [ ]:
#Análisis de las variables numéricas para verificar las transformaciones que se requieren

#Histograma de 'Page total likes'
sns.set(rc={'figure.figsize':(6,4)})
X_train_v["Page total likes"].hist(bins=18)

In [ ]:
#Diagrama de caja de 'Page total likes'
plt.boxplot(X_train_v['Page total likes'], vert=False, showmeans=True)
plt.show()

In [ ]:
#Transformación a aplicar en 'Page total likes'

ptl_max= X_train_v['Page total likes'].max()
np.power(ptl_max+ 1-X_train_v['Page total likes'], 0.1).hist(bins=20)

**Análisis**

Debido a que el histograma de 'Page total likes' muestra un sesgo negativo, además de tener un rango de valores mayor al de las otras variables de entrada, se le debe aplicar una transformación basada en su potencia.

-----

In [ ]:
#Histograma de 'Page Month'
X_train_v["Post Month"].hist(bins=23)

**Análisis**

Debido a que el rango de 'Post Month' no se expande a valores tan diferentes de las demás variables y su comportamiento es relativamente similar a una distribución normal, no requiere una transformación específica para ajustarla.

---

In [ ]:
#Histograma de 'Page Weekday'
X_train_v["Post Weekday"].hist(bins=13)

**Análisis**

Debido a que el rango de 'Post Weekday' tampoco se expande a valores tan diferentes de las demás variables y su comportamiento es relativamente similar a una distribución normal, no requiere una transformación específica para ajustarla.

---

In [ ]:
#Histograma de 'Page Hour'
X_train_v["Post Hour"].hist(bins=50)

**Análisis**

Debido a que 'Post Hour' no muestra valores de rango tan extensos y muestra un comportamiento similar a una distribución acampanada, puede dejarse de esta manera sin transformar. 

---

In [ ]:
#Histograma de 'LPE'
y_train_v['LPE'].hist(bins=20)

In [ ]:
#Transformación a aplicar en 'LPE'

np.log(1 + y_train_v['LPE']).hist(bins=20)

**Análisis**

Debido a que 'LPE' es la variable de salida, solamente se le ajusta el sesgo. No se requiere ajustar el rango.

---

In [ ]:
#Análisis de las variables categóricas para confirmar las transformaciones requeridas

#Conteo total de Valores en 'Category'
X_train_v['Category'].value_counts()

In [ ]:
#Obtención del porcentaje de cada elemento con respecto al total

X_train_v['Category'].value_counts()/X_train_v.shape[0]

**Análisis**

En el caso de la variable 'Category', se puede apreciar que cada una de las etiquetas cuenta con una representación por encima del 5% del total en el conjunto de datos, por lo cual se pueden tomar en cuenta sin necesidad de agrupar elementos. 


---

In [ ]:
#Conteo total de Valores en 'Paid'

X_train_v['Paid'].value_counts()

In [ ]:
#Obtención del porcentaje de cada elemento con respecto al total

X_train_v['Paid'].value_counts()/X_train_v.shape[0]

**Análisis**

De igual forma, en la variable 'Paid', se puede apreciar que cada una de las etiquetas cuenta con una representación por encima del 5% del total en el conjunto de datos, por lo cual se pueden tomar en cuenta sin necesidad de agrupar elementos.

---

In [ ]:
#Conteo total de Valores en 'Type'

X_train_v['Type'].value_counts()

In [ ]:
#Obtención del porcentaje de cada elemento con respecto al total

X_train_v['Type'].value_counts()/X_train_v.shape[0]

**Análisis**

Debido a que dos de las etiquetas de 'Type' se encuentran en un valor por debajo del mínimo porcentaje de representación, se van a unir en una nueva categoría para poder cumplir con el criterio del mínimo de información.

---

In [ ]:
#Definición de las funciones para transformar 'Page total likes', 'LPE' y 'Type'

#Transformador de Potencia
def mi_power(x):
  x_max=np.max(x)
  t= np.power(x_max + 1 - x, 0.1)
  return t

#Transformador de Logaritmo natural
def mi_log(y):
  t= np.log(1+y)
  return t

#Tranformado de Logaritmo natural inverso
# *Opcional, si se quisiera obtener las predicciones y la salida en unidades originales
def mi_loginv(y):
  t= np.exp(y) - 1
  return t 

#Definición de la agrupación nueva para la variable Type
def mi_type(x):
  x['Type'] = x['Type'].map({1:1, 2:2, 0:4, 3:4}) 
  return x

In [ ]:
#Transformación de los conjuntos

#Pipeline para Variables Numéricas
numericalImp_pipeline = Pipeline(steps=[('numImp', SimpleImputer(strategy='median'))])
numImp_pipe_names = ['Page total likes', 'Post Month', 'Post Weekday', 'Post Hour']

#Pipeline para Variables Numéricas
numericalPow_pipeline = Pipeline(steps=[('numPow', FunctionTransformer(mi_power))])
numPow_pipe_names = ['Page total likes']

#Pipeline para Variables Categóricas
categoricalImp_pipeline = Pipeline(steps=[('catImp', SimpleImputer(strategy='most_frequent'))])
catImp_pipe_names = ['Category', 'Paid', 'Type']

#Pipeline para Variables Categóricas
categoricalMap_pipeline = Pipeline(steps=[('catMap', FunctionTransformer(mi_type))])
catMap_pipe_names = ['Type']

#Pipeline para Variables Categóricas
categoricalOhE_pipeline = Pipeline(steps=[('catOhE', OneHotEncoder(drop='first', handle_unknown='ignore'))])
catOhE_pipe_names = ['Category', 'Paid', 'Type']

# ColumnTransformer nos permite aplicar diferentes preprocesamientos a distintas
# secciones de los datos
preprocessor = ColumnTransformer(
    transformers=[
    ('numImp', numericalImp_pipeline, numImp_pipe_names),
    ('numPow', numericalPow_pipeline, numPow_pipe_names),
    ('catImp', categoricalImp_pipeline, catImp_pipe_names),
    ('catMap', categoricalMap_pipeline, catMap_pipe_names),
    ('catOhE', categoricalOhE_pipeline, catOhE_pipe_names)
],
remainder='passthrough')

In [ ]:
#Representación gráfica de los pipeline en ColumnTransformer
preprocessor

#**Ejercicio-5.**

In [ ]:
#Partición de los datos en 100 para validación y 300 para entrenamiento
X_train, X_val, y_train, y_val = train_test_split(X_train_v, y_train_v, test_size=100, shuffle=True)

print('\033[1m' + 'Forma de X_train:' + '\033[0m', X_train.shape)
print('\033[1m' + 'Forma de y_train:' + '\033[0m', y_train.shape)
print('\033[1m' + 'Forma de X_val:' + '\033[0m', X_val.shape)

In [ ]:
#Ejecución del modelo DummyRegressor

#Fit de los datos de entrenamiento para el modelo DummyRegressor
Xtrain_ct=preprocessor.fit_transform(X_train)

#Transformación de los conjuntos de entrenamiento y validación para la variable de salida
ytrain_Log= mi_log(y_train['LPE'])
yval_Log= mi_log(y_val['LPE'])

#Preparación del modelo DummyRegressor con la estrategia 'mean'
model_DR= DummyRegressor(strategy='mean')
model_DR.fit(Xtrain_ct, ytrain_Log)

#Transformación de los datos de validación para las predicciones del modelo
Xval_ct=preprocessor.transform(X_val)

#Obtención de las predicciones del modelo usando los datos de entrenamiento y validación ajustados al DummyRegressor
yhat_DR_t= model_DR.predict(Xtrain_ct) 
yhat_DR_v= model_DR.predict(Xval_ct)

#Cálculo de los errores RMSE, MAE y MAPE en los conjuntos de entrenamiento y validación
rmse_train = mi_RMSE(ytrain_Log, yhat_DR_t)
rmse_val = mi_RMSE(yval_Log, yhat_DR_v)

mae_train = mi_MAE(ytrain_Log, yhat_DR_t)
mae_val = mi_MAE(yval_Log, yhat_DR_v)

mape_train = mi_MAPE(ytrain_Log, yhat_DR_t)
mape_val = mi_MAPE(yval_Log, yhat_DR_v)

# Imprimir los errores
print('\033[1m' + "RMSE en entrenamiento:" + '\033[0m', rmse_train)
print('\033[1m' + "RMSE en validación:" + '\033[0m', rmse_val)
print('\033[1m' + "MAE en entrenamiento:" + '\033[0m', mae_train)
print('\033[1m' + "MAE en validación:" + '\033[0m', mae_val)
print('\033[1m' + "MAPE en entrenamiento:" + '\033[0m', mape_train)
print('\033[1m' + "MAPE en validación:" + '\033[0m', mape_val)

#**Ejercicio-6.**

In [ ]:
#Definición de la función con las listas de los modelos a ejecutar

def get_models():

  #Definición inicial de las variables 'models' y 'model_names'
  models= list()
  model_names= list()

  #Modelo LR - Regresión Lineal Múltiple
  models.append(LinearRegression())
  model_names.append('LR')

  #Modelo RF - Bosque Aleatorio
  models.append(RandomForestRegressor())
  model_names.append('RF')

  #Modelo MLP - Red Neuronal Artificial / Perceptrón Lineal Multicapa
  models.append(MLPRegressor(max_iter=10000, hidden_layer_sizes=(15,), activation= 'tanh'))
  model_names.append('MLP')
  
  return models, model_names

In [ ]:
#Evaluación de los modelos LR, RF y MLP con el conjunto de datos de entrenamiento y validación

#Importado de librería 'warnings' para eliminar el desplegado de los errores
import warnings
warnings.filterwarnings('ignore')

#Carga de los modelos a comparar
models, model_names = get_models()

#Definición de la variable 'resultados'
resultados=list()

for i in range(len(models)):

  # Uso del preprocesador definido anteriormente con el modelo correspondiente para cada iteracion
  pipe= Pipeline(steps= [('ColT', preprocessor), ('m', models[i])])
  
  #Definición del tipo de validacion cruzada a utilizar
  kfold= RepeatedKFold(n_splits=5, n_repeats=3)

  #Definición de las métricas a utilizar
  mis_metricas= {'rmse':make_scorer(mi_RMSE), 'mae':make_scorer(mi_MAE), 'mape':make_scorer(mi_MAPE)}

  #Resultados obtenidos por validación cruzada
  scores= cross_validate(pipe,
                         X_train_v, mi_log(np.ravel(y_train_v)),
                         scoring= mis_metricas,
                         cv= kfold,
                         return_train_score=True,
                         n_jobs=-1,
                         error_score='raise')
  
  resultados.append(scores)

  print('%s - Train\nmean RMSE: %.4f \nmean MAE: %.3f \nmean MAPE: %.4f \n' % (model_names[i],
                                                                               np.mean(scores['train_rmse']),
                                                                               np.mean(scores['train_mae']),
                                                                               np.mean(scores['train_mape'])
                                                                               ))
  
  print('%s - Validation\nmean RMSE: %.4f \nmean MAE: %.3f \nmean MAPE: %.4f \n' % (model_names[i],
                                                                               np.mean(scores['test_rmse']),
                                                                               np.mean(scores['test_mae']),
                                                                               np.mean(scores['test_mape'])
                                                                               ))


**Conclusiones**

Con base a las respuestas obtenidas en esta parte, se pueden observar valores de métricas menores a las obtenidas por los autores del artículo de investigación.

#**Ejercicio-7.**

In [ ]:
#Valores en diagrama de caja y bigotes con la métrica "MAPE"
sns.set(rc={'figure.figsize':(8,4)})

bpMAPE_1 = list()
for i in range(len(resultados)):
  rr = resultados[i]['test_mape']
  bpMAPE_1.append(rr)

plt.boxplot(bpMAPE_1, labels=model_names, showmeans=True)
plt.show()

**Conclusiones**

Con base a los diagramas de caja y bigote anteriormente mostrados, en comparación con los resultados obtenidos por los autores del artículo,

#**Ejercicio-8.**

In [ ]:
#GridSearch para la búsqueda de los mejores hiperparámetros para el MLP

#Definición del modelo del perceptrón lineal multicapa
m_MLP= MLPRegressor(max_iter=20000)

#Definición del pipeline a utilizar
pipe2= Pipeline(steps=[('ColT', preprocessor),('mlp', m_MLP)])

#Definición del tipo de validacion cruzada a utilizar
kfold2= RepeatedKFold(n_splits=5, n_repeats=3)

#Definición de los hiperparámetros a iterar en el grid
mlp_grid_d= dict(
    mlp__hidden_layer_sizes=[(10,), (7,7), (15,), (7,8), (3,8)],
    mlp__alpha= [0.001 ,0.01, 1.0, 10.0],
    mlp__learning_rate= ['constant', 'invscaling', 'adaptive'],
    mlp__learning_rate_init=[0.01, 0.1, 1.0],
    mlp__activation= ['logistic', 'tanh']
)

#Ejecución del grid search para obtener los mejores hiperparámetros
mlp_grid = GridSearchCV(estimator=pipe2,
                        param_grid=mlp_grid_d,
                        cv=kfold2,
                        scoring= make_scorer(mi_MAPE),
                        n_jobs=-1,
                        error_score='raise')

#Grid Fit con los datos del conjunto de entrenamiento original
mlp_grid.fit(X_train_v, mi_log(y_train_v['LPE']))

#Obtención de los mejores parámetros y la mejor respuesta
print('\033[1m' + "Mejores parámetros en la búsqueda de cuadrilla (GridSearchCV):" + '\033[0m', mlp_grid.best_params_)
print('\033[1m' + "Mejor puntuación de la búsqueda de cuadrilla (GridSearchCV):" + '\033[0m', mlp_grid.best_score_)
print('\033[1m' + 'Métrica utilizada:'+ '\033[0m', mlp_grid.scoring)

**Conclusiones**

Con base a los diagramas de caja y bigote anteriormente mostrados, en comparación con los resultados obtenidos por los autores del artículo,

#**Ejercicio-9.**

In [ ]:
#Comprobación de Hiperparámetros en el modelo

#Mejores Hiperparámetros correspondientes 
model_MLP= MLPRegressor(hidden_layer_sizes=(15,),
                        activation= 'logistic',
                        max_iter=20000,
                        alpha=10.0,
                        learning_rate='invscaling',
                        learning_rate_init=1.0)
 
#Fit_transform del conjunto de entrenamiento inicial a través del pipeline
Xtvct= preprocessor.fit_transform(X_train_v)
X_testct= preprocessor.transform(X_test)

#Entrenamiento del modelo MLP
model_MLP.fit(Xtvct, mi_log(y_train_v['LPE']))

#Obtención de predicciones del modelo
yhat= model_MLP.predict(X_testct)

#Métricas Obtenidas con las predicciones
print('\033[1m' + "MLP - RMSE: %.4f"  % mi_RMSE(np.ravel(mi_log(y_test)),yhat) + '\033[0m')
print('\033[1m' + "MLP - MAE: %.4f"  % mi_MAE(np.ravel(mi_log(y_test)),yhat) + '\033[0m')
print('\033[1m' + "MLP - MAPE: %.2f %%"  % mi_MAPE(np.ravel(mi_log(y_test)),yhat) + '\033[0m')

In [ ]:
#Definición del modelo con los mejores parámetros
mMLPbest= MLPRegressor(hidden_layer_sizes=(15,),
                        activation= 'logistic',
                        max_iter=20000,
                        alpha=10.0,
                        learning_rate='invscaling',
                        learning_rate_init=1.0)

#Fit de datos de prueba en modelo con los mejores parámetros
mMLPbest.fit(X_test, mi_log(y_test['LPE']))

#Definición de la variable 'importance' para observar las características más importantes
importance= permutation_importance(mMLPbest, X_test, mi_log(y_test['LPE']), n_repeats=10)

for i,v in enumerate(importance['importances_mean']):
  print('Feature: %0d, Score: %.5f' % (i,v))

#Generación de diagrama de barras con las características más importantes
fig, axs= plt.subplots()
plt.barh([x for x in range(len(importance['importances_mean']))], importance['importances_mean'])
plt.yticks(range(X_test.shape[1]), np.array(X_test.columns))
plt.show

**Conclusiones**

Con base a los resultados obtenidos, el valor de alpha obtenido por el gridsearch no fue el que nos entregó la mejor información posible. Al contrario, aumentó el valor de las métricas, por lo que se decidió disminuirlo para comprobar el efecto.

#**Ejercicio-10.**

In [ ]:
#GridSearch para la búsqueda de los mejores hiperparámetros para el RF

#Definición del modelo de bosque aleatorio
m_RF= RandomForestRegressor()

#Definición del pipeline a utilizar
pipe3= Pipeline(steps=[('ColT', preprocessor),('rf', m_RF)])

#Definición del tipo de validacion cruzada a utilizar
kfold3= RepeatedKFold(n_splits=5, n_repeats=3)

#Definición de los hiperparámetros a iterar en el grid
rf_grid_d= [
    {
     'rf__max_depth':[2,3,4,5,6],
     'rf__min_samples_split': [2,3,4,5,6,7],
     'rf__ccp_alpha' : [0.001, 0.01, 0.1]
    }
]

#Ejecución del grid search para obtener los mejores hiperparámetros
rf_grid = GridSearchCV(estimator=pipe3,
                        param_grid=rf_grid_d,
                        cv=kfold3,
                        scoring= make_scorer(mi_MAPE),
                        n_jobs=-1,
                        error_score='raise')

#Grid Fit con los datos del conjunto de entrenamiento original
rf_grid.fit(X_train_v, mi_log(y_train_v['LPE']))

#Obtención de los mejores parámetros y la mejor respuesta
print('\033[1m' + "Mejores parámetros en la búsqueda de cuadrilla (GridSearchCV):" + '\033[0m', rf_grid.best_params_)
print('\033[1m' + "Mejor puntuación de la búsqueda de cuadrilla (GridSearchCV):" + '\033[0m', rf_grid.best_score_)
print('\033[1m' + 'Métrica utilizada:'+ '\033[0m', rf_grid.scoring)

In [ ]:
#Comprobación de Hiperparámetros en el modelo

#Mejores Hiperparámetros correspondientes 
model_RF= RandomForestRegressor(ccp_alpha=0.1,
                                max_depth= 6,
                                min_samples_split=6,
)

#Fit_transform del conjunto de entrenamiento inicial a través del pipeline
Xtvct= preprocessor.fit_transform(X_train_v)
X_testct= preprocessor.transform(X_test)

#Entrenamiento del modelo RF
model_RF.fit(Xtvct, mi_log(y_train_v))

#Obtención de predicciones del modelo
yhat=model_RF.predict(X_testct)

#Métricas Obtenidas con las predicciones
print('\033[1m' + "RF - RMSE: %.4f"  % mi_RMSE(np.ravel(mi_log(y_test)),yhat) + '\033[0m')
print('\033[1m' + "RF - MAE: %.4f"  % mi_MAE(np.ravel(mi_log(y_test)),yhat) + '\033[0m')
print('\033[1m' + "RF - MAPE: %.2f %%"  % mi_MAPE(np.ravel(mi_log(y_test)),yhat) + '\033[0m')

In [ ]:
#Definición del modelo con los mejores parámetros
mRFbest= RandomForestRegressor(ccp_alpha=0.1, max_depth= 5, min_samples_split=5,)

#Fit de datos de prueba en modelo con los mejores parámetros
mRFbest.fit(X_test, mi_log(y_test['LPE']))

#Definición de la variable 'importance' para observar las características más importantes
importance= permutation_importance(mRFbest, X_test, mi_log(y_test['LPE']), n_repeats=10)

for i,v in enumerate(importance['importances_mean']):
  print('Feature: %0d, Score: %.5f' % (i,v))

#Generación de diagrama de barras con las características más importantes
fig, axs= plt.subplots()
plt.barh([x for x in range(len(importance['importances_mean']))], importance['importances_mean'])
plt.yticks(range(X_test.shape[1]), np.array(X_test.columns))
plt.show

**Conclusiones**

Con base a los resultados obtenidos , 

#**Ejercicio-11.**

In [ ]:
#GridSearch para la búsqueda de los mejores hiperparámetros para el LR

#Definición del modelo de regresión lineal múltiple
m_LR= LinearRegression()

#Definición del pipeline a utilizar
pipe4= Pipeline(steps=[('ColT', preprocessor),('lr', m_LR)])

#Definición del tipo de validacion cruzada a utilizar
kfold4= RepeatedKFold(n_splits=5, n_repeats=3)

#Definición de los hiperparámetros a iterar en el grid
lr_grid_d= [
    {
     'lr__fit_intercept': [True, False],
     'lr__copy_X': [True, False],
     'lr__positive': [True, False]
    }
]

#Ejecución del grid search para obtener los mejores hiperparámetros
lr_grid = GridSearchCV(estimator=pipe4,
                        param_grid=lr_grid_d,
                        cv=kfold4,
                        scoring= make_scorer(mi_MAPE),
                        n_jobs=-1,
                        error_score='raise')

#Grid Fit con los datos del conjunto de entrenamiento original
lr_grid.fit(X_train_v, mi_log(y_train_v['LPE']))

#Obtención de los mejores parámetros y la mejor respuesta
print('\033[1m' + "Mejores parámetros en la búsqueda de cuadrilla (GridSearchCV):" + '\033[0m', lr_grid.best_params_)
print('\033[1m' + "Mejor puntuación de la búsqueda de cuadrilla (GridSearchCV):" + '\033[0m', lr_grid.best_score_)
print('\033[1m' + 'Métrica utilizada:'+ '\033[0m', lr_grid.scoring)

In [ ]:
#Comprobación de Hiperparámetros en el modelo

#Hiperparámetros correspondientes 
model_LR= LinearRegression(copy_X=True,
                           fit_intercept=False,
                           positive=True    
)

#Fit_transform del conjunto de entrenamiento inicial a través del pipeline
Xtvct= preprocessor.fit_transform(X_train_v)
X_testct= preprocessor.transform(X_test)

#Entrenamiento del modelo LR
model_LR.fit(Xtvct, mi_log(y_train_v))

#Obtención de predicciones del modelo
yhat=model_LR.predict(X_testct)

#Métricas Obtenidas con las predicciones
print('\033[1m' + "LR - RMSE: %.4f"  % mi_RMSE((mi_log(y_test)),yhat) + '\033[0m')
print('\033[1m' + "LR - MAE: %.4f"  % mi_MAE((mi_log(y_test)),yhat) + '\033[0m')
print('\033[1m' + "LR - MAPE: %.2f %%"  % mi_MAPE((mi_log(y_test)),yhat) + '\033[0m')

In [ ]:
#Definición del modelo con los mejores parámetros
mLRbest= LinearRegression()

#Fit de datos de prueba en modelo con los mejores parámetros
mLRbest.fit(X_test, mi_log(y_test['LPE']))

#Definición de la variable 'importance' para observar las características más importantes
importance= permutation_importance(mLRbest, X_test, mi_log(y_test['LPE']), n_repeats=10)

for i,v in enumerate(importance['importances_mean']):
  print('Feature: %0d, Score: %.5f' % (i,v))

#Generación de diagrama de barras con las características más importantes
fig, axs= plt.subplots()
plt.barh([x for x in range(len(importance['importances_mean']))], importance['importances_mean'])
plt.yticks(range(X_test.shape[1]), np.array(X_test.columns))
plt.show

**Conclusiones**

Con base a los resultados obtenidos , 

#**Ejercicio-12.**

**Conclusiones**

Con base a los resultados obtenidos en esta actividad en comparación con los obtenidos por los autores del artículo de Moro-Rita-Vala, 

###**Fin de la Actividad de la semana 7.**